In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing 
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.utils import shuffle
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

2024-08-18 12:51:46.495742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 12:51:46.495854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 12:51:46.630849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

/tmp/ipykernel_25/2653809078.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [3]:
df_2018 = pd.read_csv('/kaggle/input/isic-2018-jpg-224x224-resized/train-metadata.csv')
df_2019 = pd.read_csv('/kaggle/input/isic-2019-jpg-224x224-resized/train-metadata.csv')
df_2020 = pd.read_csv('/kaggle/input/isic-2020-jpg-224x224-resized/train-metadata.csv')

In [4]:
# import shutil
# import os
# def create_path(dir):
#     if not os.path.exists(dir):
#         os.makedirs(dir)
        
# create_path('/kaggle/working/images/positive')
# def process(isic_id, src, dst):
#     # Construct the full file path
#     src_file = os.path.join(src, isic_id)  # Assuming the images have a .jpg extension
#     dst_file = dst
    
#     # Check if the source file exists and move it
#     if os.path.exists(src_file):
#         try:
#             shutil.move(src_file, dst_file)
#         except Exception as e:
#             print(e)

# # Define source and destination directories
# src_dir = '/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image'
# dst_dir = '/kaggle/working/images/positive'

# # Apply the process function to each DataFrame and filter out the successfully moved files
# df_2018_positive['moved'] = df_2018_positive['isic_id'].apply(lambda x: process(x, '/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image', dst_dir))
# df_2019_positive['moved'] = df_2019_positive['isic_id'].apply(lambda x: process(x, src_dir, dst_dir))
# df_2020_positive['moved'] = df_2020_positive['isic_id'].apply(lambda x: process(x, '/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image', dst_dir))

# # Filter the rows with moved files and concatenate them to the main DataFrame
# df_image = pd.concat([df_image, 
#                 df_2018_positive[df_2018_positive['moved']], 
#                 df_2019_positive[df_2019_positive['moved']], 
#                 df_2020_positive[df_2020_positive['moved']]
#                ]).reset_index(drop=True)

# # Optionally drop the 'moved' column if you don't need it
# df_image.drop(columns=['moved'], inplace=True)


## Constants

In [5]:
RANDOM_STATE = 42
IMAGE_SIZE=224
BATCH_SIZE = 64

# img_augmentation_layers = [
#     layers.RandomRotation(factor=0.15),
#     layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
#     layers.RandomFlip(),
#     layers.RandomContrast(factor=0.1),
# ]




## Helper functions

In [6]:
def get_sample(df,ratio):
    df_positive = df[df['target']==1].reset_index(drop=True)
    df_negative = df[df['target']==0].reset_index(drop=True)
    df_negative= resample(df_negative,n_samples=len(df_positive)*ratio,random_state=RANDOM_STATE)
#     df_positive = resample(df_positive,n_samples=len(df_positive),random_state=RANDOM_STATE)
    
    df_concat = pd.concat([df_positive,df_negative])
    df_concat = shuffle(df_concat).reset_index(drop=True)
    
    print(f'positive shape: {df_positive.shape}, negative shape: {df_negative.shape}')
    print(f'Full dataset shape: {df_concat.shape}')
    
    return df_concat

def visualize(df,idx):
    images = []
    labels = []
    for id in idx:
        current_row = df.iloc[id]
        images.append(plt.imread(current_row["file_path"]))
        labels.append(current_row["target"])
    
    for i, (image, label) in enumerate(zip(images, labels), start=1):
        plt.subplot(3, 3, i)  # Adjusted to start from 1
        plt.imshow(image)
        plt.axis("off")
        plt.title(label)
    plt.show()

In [7]:
def calculate_weight(pos,neg):

    # Total number of samples
    total = pos + neg

    # Calculate class weights
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    print('Weight for class 0 (Negative): {:.2f}'.format(weight_for_0))
    print('Weight for class 1 (Positive): {:.2f}'.format(weight_for_1))
    return class_weight

In [8]:
import gc
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

clear_memory = ClearMemory()


In [9]:
class PartialAUC(tf.keras.metrics.Metric):
    def __init__(self, name='partial_auc', **kwargs):
        super(PartialAUC, self).__init__(name=name, **kwargs)
        self.thresholds = tf.linspace(0.0, 1.0, 101)
        self.tp = self.add_weight(name='tp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fp = self.add_weight(name='fp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fn = self.add_weight(name='fn', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.tn = self.add_weight(name='tn', shape=(101,), initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(tf.cast(y_true, tf.float32), (-1, 1))
        y_pred = tf.cast(y_pred, tf.float32)

        # Debugging shapes
        #tf.print("reshaped y_true shape:", tf.shape(y_true))
        #tf.print("y_pred shape:", tf.shape(y_pred))

        thresholds = self.thresholds[tf.newaxis, :]
        preds = tf.cast(y_pred[:, tf.newaxis] >= thresholds, tf.float32)

        tp = tf.reduce_sum(preds * y_true[:, tf.newaxis], axis=0)
        fp = tf.reduce_sum(preds * (1 - y_true)[:, tf.newaxis], axis=0)
        fn = tf.reduce_sum((1 - preds) * y_true[:, tf.newaxis], axis=0)
        tn = tf.reduce_sum((1 - preds) * (1 - y_true)[:, tf.newaxis], axis=0)

        # Check tensor shapes
        #tf.print("tp shape:", tf.shape(tp))
        #tf.print("fp shape:", tf.shape(fp))
        #tf.print("fn shape:", tf.shape(fn))
        #tf.print("tn shape:", tf.shape(tn))

        # Reshape assuming desired shape is (num_thresholds,)
        num_thresholds = tf.shape(self.thresholds)[0]
        
        reshaped_tp = tf.reshape(tp, [num_thresholds])
        reshaped_fp = tf.reshape(fp, [num_thresholds])
        reshaped_fn = tf.reshape(fn, [num_thresholds])
        reshaped_tn = tf.reshape(tn, [num_thresholds])
        
        self.tp.assign_add(reshaped_tp)
        self.fp.assign_add(reshaped_fp)
        self.fn.assign_add(reshaped_fn)
        self.tn.assign_add(reshaped_tn)

    def result(self):
        # Avoid division by zero
        epsilon = tf.keras.backend.epsilon()
        tpr = self.tp / (self.tp + self.fn + epsilon)
        fpr = self.fp / (self.fp + self.tn + epsilon)

        # Sort fpr and corresponding tpr
        sorted_indices = tf.argsort(fpr)
        sorted_fpr = tf.gather(fpr, sorted_indices)
        sorted_tpr = tf.gather(tpr, sorted_indices)

        # Apply mask for TPR >= 0.8
        mask = sorted_tpr >= 0.8
        partial_fpr = tf.boolean_mask(sorted_fpr, mask)
        partial_tpr = tf.boolean_mask(sorted_tpr, mask)

        # Calculate partial AUC using the trapezoidal rule
        def compute_partial_auc():
            delta_fpr = partial_fpr[1:] - partial_fpr[:-1]
            tpr_sum = partial_tpr[1:] + partial_tpr[:-1]
            return tf.reduce_sum(delta_fpr * tpr_sum / 2)

        partial_auc = tf.cond(tf.shape(partial_fpr)[0] > 1,
                              compute_partial_auc,
                              lambda: tf.constant(0.0))

        return partial_auc * 0.2

    def reset_states(self):
        self.tp.assign(tf.zeros_like(self.tp))
        self.fp.assign(tf.zeros_like(self.fp))
        self.fn.assign(tf.zeros_like(self.fn))
        self.tn.assign(tf.zeros_like(self.tn))

In [10]:
df_2018 = get_sample(df_2018,ratio=10)
df_2018['target'] = df_2018['target'].astype(str)
df_2018['isic_id'] = df_2018['isic_id'] + ".jpg"

positive shape: (785, 4), negative shape: (7850, 4)
Full dataset shape: (8635, 4)


In [11]:
train_datagen_2018 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2
)

train_2018 = train_datagen_2018.flow_from_dataframe(df_2018,
                                              directory="/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image",
                                              x_col="isic_id",
                                              y_col="target",
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              batch_size=BATCH_SIZE,
                                              class_mode='binary',
                                              )
val_2018 = train_datagen_2018.flow_from_dataframe(
    df_2018,
    directory="/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 7205 validated image filenames belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1430 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(


Found 1441 validated image filenames belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1430 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(


In [12]:
df_2019 = get_sample(df_2019,ratio=4)
df_2019['target'] = df_2019['target'].astype(str)
df_2019['isic_id'] = df_2019['isic_id'] + ".jpg"

positive shape: (4522, 4), negative shape: (18088, 4)
Full dataset shape: (22610, 4)


In [13]:
train_datagen_2019 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)
# Training set
train_2019 = train_datagen_2019.flow_from_dataframe(
    df_2019,
    directory="/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify that this is the training set
)

val_2019 = train_datagen_2019.flow_from_dataframe(
    df_2019,
    directory="/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 18088 validated image filenames belonging to 2 classes.
Found 4522 validated image filenames belonging to 2 classes.


In [14]:
df_2020 = get_sample(df_2020,ratio=5)

positive shape: (584, 4), negative shape: (2920, 4)
Full dataset shape: (3504, 4)


In [15]:
train_datagen_2020 = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)
df_2020['target'] = df_2020['target'].astype(str)
df_2020['isic_id'] = df_2020['isic_id'] + ".jpg" \

train_2020 = train_datagen_2020.flow_from_dataframe(
    df_2020,
    directory="/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify that this is the training set
)

# Validation set
val_2020 = train_datagen_2020.flow_from_dataframe(
    df_2020,
    directory="/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 2804 validated image filenames belonging to 2 classes.
Found 700 validated image filenames belonging to 2 classes.


In [16]:
df = get_sample(df,ratio=20)
df['target'] = df['target'].astype(str)
df['isic_id'] = df['isic_id'] + ".jpg" 
train_datagen = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)

train_data = train_datagen.flow_from_dataframe(
    df,
    directory="/kaggle/input/isic-2024-challenge/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training' 
)

positive shape: (393, 55), negative shape: (7860, 55)
Full dataset shape: (8253, 55)
Found 6603 validated image filenames belonging to 2 classes.


## Vision Model

In [17]:
from keras.applications import EfficientNetB0

In [18]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="/kaggle/input/efficient-net-b0-no-top-weights/efficientnetb0_notop.h5")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(512, activation="relu")(x)
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation="sigmoid", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=[PartialAUC()]
    )
    return model

In [19]:
strategy = tf.distribute.MirroredStrategy()

# Use the strategy to scope model creation and training
with strategy.scope():
    eff_model = build_model(num_classes=1)

In [20]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist_2018 = eff_model.fit(train_2018,validation_data=val_2018, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5


2024-08-18 12:53:20.070281: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_779/cond/StatefulPartitionedCall/EfficientNet_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 181s 1s/step - loss: 1.2296 - partial_auc: 0.0212 - val_loss: 0.3884 - val_partial_auc: 0.0000e+00
Epoch 2/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - loss: 0.3476 - partial_auc: 0.0339 - val_loss: 0.2937 - val_partial_auc: 0.0000e+00
Epoch 3/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - loss: 0.3631 - partial_auc: 0.0377 - val_loss: 0.2997 - val_partial_auc: 0.0000e+00
Epoch 4/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - loss: 0.3302 - partial_auc: 0.0258 - val_loss: 0.2995 - val_partial_auc: 0.0000e+00
Epoch 5/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - loss: 0.3286 - partial_auc: 0.0183 - val_loss: 0.3100 - val_partial_auc: 0.0000e+00


In [21]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist_2019 = eff_model.fit(train_2019,validation_data=val_2019, epochs=2,callbacks=[clear_memory])

Epoch 1/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 399s 1s/step - loss: 0.5218 - partial_auc: 0.0264 - val_loss: 0.5035 - val_partial_auc: 0.0000e+00
Epoch 2/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - loss: 0.4907 - partial_auc: 1.1391e-04 - val_loss: 0.5045 - val_partial_auc: 0.0000e+00


In [22]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist_2020 = eff_model.fit(train_2020,validation_data=val_2020, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - loss: 0.4467 - partial_auc: 0.0025 - val_loss: 0.4800 - val_partial_auc: 0.0000e+00
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 0.4538 - partial_auc: 7.3895e-04 - val_loss: 0.4810 - val_partial_auc: 0.0000e+00
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - loss: 0.4611 - partial_auc: 0.0000e+00 - val_loss: 0.4869 - val_partial_auc: 0.0000e+00
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 0.4437 - partial_auc: 0.0000e+00 - val_loss: 0.4629 - val_partial_auc: 0.0000e+00
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 0.4378 - partial_auc: 0.0000e+00 - val_loss: 0.4767 - val_partial_auc: 0.0000e+00


In [23]:
hist_2020 = eff_model.fit(train_data, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 111s 994ms/step - loss: 0.2295 - partial_auc: 0.0140
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 93s 834ms/step - loss: 0.1956 - partial_auc: 0.0043
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 843ms/step - loss: 0.1931 - partial_auc: 1.2499e-04
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 840ms/step - loss: 0.1842 - partial_auc: 0.0019
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 93s 837ms/step - loss: 0.1928 - partial_auc: 1.5556e-04


In [24]:
len(eff_model.layers)

246

In [25]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-50:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=[PartialAUC()]
    )
    return model

with strategy.scope():
    eff_model = unfreeze_model(eff_model)

In [26]:
eff_model.summary()

Model: "EfficientNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_

 Total params: 6,055,588 (23.10 MB)

 Trainable params: 4,040,977 (15.42 MB)

 Non-trainable params: 2,014,611 (7.69 MB)

In [27]:
hist_2018 = eff_model.fit(train_2018,validation_data=val_2018, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5


2024-08-18 13:29:31.747109: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_859/cond/StatefulPartitionedCall/EfficientNet_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


113/113 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - loss: 0.3222 - partial_auc: 0.0101 - val_loss: 0.3178 - val_partial_auc: 0.0000e+00
Epoch 2/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - loss: 0.3005 - partial_auc: 0.0017 - val_loss: 0.2810 - val_partial_auc: 0.0000e+00
Epoch 3/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - loss: 0.3158 - partial_auc: 4.7343e-04 - val_loss: 0.3061 - val_partial_auc: 0.0000e+00
Epoch 4/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - loss: 0.3156 - partial_auc: 0.0000e+00 - val_loss: 0.3064 - val_partial_auc: 0.0000e+00
Epoch 5/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - loss: 0.2925 - partial_auc: 0.0000e+00 - val_loss: 0.3093 - val_partial_auc: 0.0000e+00


In [28]:
hist_2020 = eff_model.fit(train_data, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 843ms/step - loss: 0.1990 - partial_auc: 0.0060
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 92s 824ms/step - loss: 0.1919 - partial_auc: 4.3127e-04
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 95s 853ms/step - loss: 0.1991 - partial_auc: 0.0020
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 93s 835ms/step - loss: 0.1927 - partial_auc: 0.0000e+00
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 840ms/step - loss: 0.1963 - partial_auc: 0.0033


In [29]:
tb_callback = tf.keras.callbacks.TensorBoard('./kaggle/working/', update_freq=1)

In [30]:
epochs = 20 
hist = eff_model.fit(train_data, epochs=epochs,callbacks=[clear_memory,tb_callback])

Epoch 1/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 96s 845ms/step - loss: 0.1787 - partial_auc: 4.2911e-04
Epoch 2/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 844ms/step - loss: 0.1923 - partial_auc: 4.2256e-04
Epoch 3/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 95s 856ms/step - loss: 0.1879 - partial_auc: 0.0000e+00
Epoch 4/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 96s 866ms/step - loss: 0.1890 - partial_auc: 0.0000e+00
Epoch 5/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 841ms/step - loss: 0.2039 - partial_auc: 0.0016
Epoch 6/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 844ms/step - loss: 0.1905 - partial_auc: 6.1689e-04
Epoch 7/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 95s 848ms/step - loss: 0.1989 - partial_auc: 3.2376e-04
Epoch 8/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 93s 836ms/step - loss: 0.1880 - partial_auc: 0.0000e+00
Epoch 9/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 94s 846ms/step - loss: 0.1893 - partial_auc: 0.0000e+00
Epoch 10/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 93s 836ms/step - loss: 0.1913 - partial_auc: 0.0000e+00
Epoch 11/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 93

In [31]:
# strategy = tf.distribute.MirroredStrategy()

# # Use the strategy to scope model creation and training
# with strategy.scope():
#     model = vision_model()

In [32]:
def create_vision_encoder(trainable_layer, trainable=False, input_shape=(139,139,3)):
    # Load ResNet50V2 model with lower computational complexity
    model = keras.applications.ResNet50V2(include_top=False, weights='/kaggle/input/resnet-weights/resnet50v2_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=input_shape)
    
    # Freeze all layers by default
    model.trainable = False
    
    # Optionally unfreeze layers up to trainable_layer
    if trainable:
        for layer in model.layers[-10:]:
            layer.trainable = trainable
    
    # Define input and apply preprocessing
    image = keras.Input(shape=input_shape, name="image")
    preprocessed = keras.applications.resnet_v2.preprocess_input(image)
    
    # Get the output from the model
    outputs = model(preprocessed)
    
    # Return the final model
    return keras.Model(inputs=[image], outputs=outputs, name="vision_encoder")


In [33]:
def vision_model():
    vision_encoder = create_vision_encoder(trainable_layer=235,input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),trainable=True)
    
    x = layers.MaxPooling2D()(vision_encoder.output)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64,activation='relu')(x)
    x = layers.Dense(1,activation='sigmoid')(x)

    vision_model = keras.Model(inputs = vision_encoder.input, outputs=x)
    vision_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[PartialAUC()])
    return vision_model

In [34]:
# strategy = tf.distribute.MirroredStrategy()

# # Use the strategy to scope model creation and training
# with strategy.scope():
#     model = vision_model()

In [35]:
# model

In [36]:
# history = model.fit(train_data, epochs=10)

In [37]:
import h5py


In [38]:
hdf5_file_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'

In [39]:
hdf5_file = h5py.File(hdf5_file_path, 'r')

def preprocess_image(image_bytes):
    image = tf.io.decode_jpeg(image_bytes)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

image_list = []
image_ids = []

for key in hdf5_file.keys():
    image_bytes = hdf5_file[key][()]
    image = preprocess_image(image_bytes)
    
    image_list.append(image)
    image_ids.append(key)
images_tensor = tf.stack(image_list)

# You now have the images ready for prediction
# Example: predictions = model.predict(images_tensor)

# Make sure to close the file
hdf5_file.close()

https://github.com/sayakpaul/Multimodal-Entailment-Baseline/blob/main/multimodal_entailment.ipynb

In [40]:
predictions = eff_model.predict(images_tensor)
predicted_classes = np.argmax(predictions, axis=1)
submission = pd.DataFrame({
    'isic_id': image_ids,
    'target': predicted_classes
})

submission.to_csv('submission.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
